### Install Dependencies

In [ ]:
# Might need to install tesseract into your system separately
# !pip install pytesseract

In [1]:
!pip install easyocr

     |████████████████████████████████| 48.4 MB 11.4 MB/s eta 0:00:01   |█▏                              | 1.7 MB 561 kB/s eta 0:01:24     |██▉                             | 4.3 MB 5.8 MB/s eta 0:00:08     |██████▌                         | 9.8 MB 5.8 MB/s eta 0:00:07     |██████████████████████████████▋ | 46.3 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 52.2 MB 10.3 MB/s eta 0:00:01   |█████▏                          | 8.4 MB 9.4 MB/s eta 0:00:05     |█████████████████████▍          | 34.9 MB 9.6 MB/s eta 0:00:02
     |████████████████████████████████| 387 kB 12.8 MB/s eta 0:00:01


### Write Text Detection Class

In [2]:
import easyocr
reader = easyocr.Reader(['en']) # need to run only once to load model into memory
result = reader.readtext('images/plate_number.jpg')

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [3]:
result

[([[0, 390], [78, 390], [78, 444], [0, 444]], 'NA', 0.9352347254753113),
 ([[85, 459], [113, 459], [113, 479], [85, 479]], 'LNL', 0.05920890346169472),
 ([[82, 460], [212, 460], [212, 508], [82, 508]],
  'PhPHUHFIP',
  0.00015153623826336116),
 ([[0, 489], [75, 489], [75, 529], [0, 529]], 'ALA', 0.5165254473686218),
 ([[85, 495], [170, 495], [170, 523], [85, 523]],
  'Pharmacy',
  0.844280481338501),
 ([[411, 542], [534, 542], [534, 605], [411, 605]], 'TGP', 0.5467938184738159),
 ([[407, 606], [551, 606], [551, 627], [407, 627]],
  'THE GJNEBICS PHARMICV',
  0.0003282821271568537),
 ([[916, 616], [979, 616], [979, 634], [916, 634]],
  'Generlsa',
  0.3196142613887787),
 ([[222, 650], [272, 650], [272, 700], [222, 700]],
  'Lil',
  0.06678491085767746),
 ([[101, 718], [229, 718], [229, 736], [101, 736]],
  'Ausonomaus Urlyrrsity',
  0.019428178668022156),
 ([[111, 733], [223, 733], [223, 749], [111, 749]],
  'as gran!ed byLHgD',
  0.004948889371007681),
 ([[142, 762], [178, 762], [178, 

In [11]:
# result without text
result = reader.readtext('images/no_text_image1.jpg')

In [13]:
result == []

True

In [14]:
import easyocr
class TextDetectionFromImage:
    """
    Simple python class to detet images from object
    """
    def __init__(self):
        self.reader = easyocr.Reader(['en']) # need to run only once to load model into memory    

    def detect_text(self, im):
        result = self.reader.readtext(im)
        text_exists = result != []
          
        return {'has_text':text_exists, 'text_and_location': result}

text_detector = TextDetectionFromImage()

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [15]:
%%time result = text_detector.detect_text('images/plate_number.jpg')

### Write service file

In [17]:
%%writefile text_detection_service.py

from bentoml import BentoService, api, env, artifacts
from bentoml.artifact import PickleArtifact
from bentoml.adapters import FileInput

@artifacts([PickleArtifact('model')])
@env(pip_dependencies=['easyocr'], 
     conda_channels=["conda-forge"], 
     conda_dependencies=["ruamel.yaml"])
class TextDetectionService(BentoService):
    
    @api(input=FileInput())
    def predict(self, image):
        result = self.artifacts.model.detect_text(image[0])
        return result

Overwriting text_detection_service.py


### Package Model and Serve

In [18]:
# 1) import the custom BentoService defined above
from text_detection_service import TextDetectionService

# 2) `pack` it with required artifacts
bento_service = TextDetectionService()
bento_service.pack("model", text_detector)

# # 3) save your BentoService to file archive
saved_path = bento_service.save()

[2020-09-18 08:49:18,765] INFO - BentoService bundle 'TextDetectionService:20200918084916_D545CB' saved to: /Users/xavierpuspus/bentoml/repository/TextDetectionService/20200918084916_D545CB


In [19]:
saved_path

'/Users/xavierpuspus/bentoml/repository/TextDetectionService/20200918084916_D545CB'

In [7]:
from bentoml import load

service = load(saved_path)

print(service.predict('images/japan72.jpg'))

[2020-09-11 16:04:04,631] WARNING - Module `text_detection_service` already loaded, using existing imported module.
{'has_text': 'True', 'text': '72 100° }Ln;~)QV;ea=)| e2774'}


In [8]:
print(service.predict('images/plate_number.jpg'))

{'has_text': 'True', 'text': 'neral na plate number...'}


In [9]:
!pip install {saved_path}

Processing /Users/xavierpuspus/bentoml/repository/TextDetectionService/20200911160359_13C4A9
  Created wheel for TextDetectionService: filename=TextDetectionService-20200911160359_13C4A9-py3-none-any.whl size=4755 sha256=a94afe43a5460760558520902bddf1a8f970bfbe6b4016b50194aac3390884e6
  Stored in directory: /private/var/folders/83/g17gnwks0cs_007kl7c03mdr0000gn/T/pip-ephem-wheel-cache-iwlf5vog/wheels/4e/77/44/83d90a849df21517ea00011bcf0476f08a58b6a1ce38aa1df1
Successfully built TextDetectionService


In [12]:
# Ultracold class published as a package
import TextDetectionService

text_detect = TextDetectionService.load()
text_detect.predict('images/plate_number.jpg')

{'has_text': 'True', 'text': 'neral na plate number...'}

### Docker Build

!cd {saved_path} && docker build -t text-detector .